# F1 Dashboard 

This section demonstrates how to build an interactive dashboard for F1 data.

In [ ]:
# Import required libraries
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

## Weather Dashboard with Session Metadata

We'll join the weather data with session metadata, filter by date, and create a new session ID for richer dashboard context.

In [ ]:
# Load session metadata and join with weather data
sessions = pd.read_csv('../data/cleaned/sessions.csv')
# Ensure date column is datetime and sort by date ascending
if 'date' in sessions.columns:
    sessions['date'] = pd.to_datetime(sessions['date'], errors='coerce')
    sessions = sessions.sort_values('date', ascending=True)
# Get session_keys in date order
ordered_session_keys = sessions['session_key'].tolist()
weather = pd.read_csv('../data/cleaned/weather.csv')

# Filter by date (2024 only)
weather['date'] = pd.to_datetime(weather['date'], errors='coerce')
weather = weather[(weather['date'] >= '2024-01-01') & (weather['date'] <= '2024-12-31')].copy()

# Join on session_key
weather_sessions = weather.merge(sessions, on='session_key', how='left', suffixes=('', '_session'))

# Create new session_id: <session_key>-<session_name>-<circuit_short_name>
if 'session_name' in weather_sessions.columns and 'circuit_short_name' in weather_sessions.columns:
    weather_sessions['session_id'] = weather_sessions['session_key'].astype(str) + '-' + \
        weather_sessions['session_name'].astype(str) + '-' + \
        weather_sessions['circuit_short_name'].astype(str)
else:
    weather_sessions['session_id'] = weather_sessions['session_key'].astype(str)

# Prepare dropdown options for session_id in date order
session_id_ordered = []
for sk in ordered_session_keys:
    subset = weather_sessions[weather_sessions['session_key'] == sk]
    if not subset.empty:
        session_id = subset['session_id'].iloc[0]
        if session_id not in session_id_ordered:
            session_id_ordered.append(session_id)
session_options = [
    {"label": sid, "value": sid} for sid in session_id_ordered
]

# Compute aggregate statistics for each session_id
weather_agg = weather_sessions.groupby('session_id').agg({
    'air_temperature': ['mean', 'min', 'max'],
    'track_temperature': ['mean', 'min', 'max'],
    'humidity': ['mean', 'min', 'max'],
    'pressure': ['mean', 'min', 'max'],
    'rainfall': 'sum',
    'wind_speed': ['mean', 'min', 'max']
}).reset_index()
weather_agg.columns = ['_'.join(col).strip('_') for col in weather_agg.columns.values]

In [ ]:
# Weather dashboard app with improved table, variable filter, and units
app = Dash(__name__)

variable_options = [
    {"label": "Air Temperature (°C)", "value": "air_temperature"},
    {"label": "Track Temperature (°C)", "value": "track_temperature"},
    {"label": "Humidity (%)", "value": "humidity"},
    {"label": "Pressure (mbar)", "value": "pressure"},
    {"label": "Rainfall (mm)", "value": "rainfall"},
    {"label": "Wind Speed (m/s)", "value": "wind_speed"},
    {"label": "Wind Direction (°)", "value": "wind_direction"},
]

# Map variable to units for table
variable_units = {
    'air_temperature': '°C',
    'track_temperature': '°C',
    'humidity': '%',
    'pressure': 'mbar',
    'rainfall': 'mm',
    'wind_speed': 'm/s',
    'wind_direction': '°',
}

app.layout = html.Div([
    html.H1('F1 Weather Dashboard (with Session Metadata)'),
    dcc.Dropdown(
        id='session-dropdown',
        options=session_options,
        value=weather_sessions['session_id'].unique()[0],
        clearable=False,
        style={'width': '60%'}
    ),
    dcc.Dropdown(
        id='variable-dropdown',
        options=variable_options,
        value='air_temperature',
        clearable=False,
        style={'width': '40%', 'marginTop': 10, 'marginBottom': 10}
    ),
    html.Div(id='weather-agg-table', style={'width': '40%', 'margin': 'auto', 'marginBottom': 20}),
    dcc.Graph(id='weather-variable-graph')
])

@app.callback(
    Output('weather-variable-graph', 'figure'),
    Output('weather-agg-table', 'children'),
    Input('session-dropdown', 'value'),
    Input('variable-dropdown', 'value')
)
def update_weather_dashboard(selected_session_id, selected_variable):
    filtered = weather_sessions[weather_sessions['session_id'] == selected_session_id]
    unit = variable_units.get(selected_variable, '')
    # Use a simple line plot (no regression line)
    fig = px.line(
        filtered,
        x='date',
        y=selected_variable,
        title=f'{selected_variable.replace("_", " ").title()} Over Time ({selected_session_id})',
        labels={selected_variable: f'{selected_variable.replace("_", " ").title()} ({unit})', 'date': 'Date'}
    )
    # Aggregate stats for selected session_id and variable
    agg_row = weather_agg[weather_agg['session_id'] == selected_session_id]
    if not agg_row.empty:
        # Find columns for this variable
        cols = [c for c in agg_row.columns if c.startswith(selected_variable)]
        col_labels = [c.split('_')[-1].capitalize() for c in cols]
        values = [agg_row[c].values[0] for c in cols]
        stats_table = html.Table([
            html.Tr([html.Th('Statistic'), html.Th(f'Value ({unit})')]),
            *[html.Tr([html.Td(label), html.Td(f"{val:.2f}")]) for label, val in zip(col_labels, values)]
        ], style={'margin': 'auto', 'border': '1px solid #ccc', 'borderCollapse': 'collapse', 'width': '100%'})
    else:
        stats_table = html.Div('No aggregate data available.')
    return fig, stats_table

app.run(jupyter_mode='inline')